In [1]:
import os
import torch
import torchvision
import tarfile
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Subset
import torchvision.transforms as tt
from torch.utils.data import random_split
from torchvision.utils import make_grid
import matplotlib
import matplotlib.pyplot as plt
import torch.optim as optim
import random
from PIL import Image
import seaborn as sn
import pandas as pd
from torchvision.transforms import ToTensor, Normalize, Compose
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

%matplotlib inline


In [2]:
project_name='rock-paper-scissors'

In [3]:
    
# Look into the data directory
data_dir = './datasets/Datasets/Datasets'
print(os.listdir(data_dir))
classes = os.listdir(data_dir + "/train")
print(classes)

['test', 'train', 'val']
['paper', 'rock', 'scissors']


In [4]:
stats=((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

train_tfms = tt.Compose([tt.CenterCrop(224),
                         tt.Resize((32,32)), 
                         tt.RandomHorizontalFlip(), 
                         tt.ToTensor(), 
                         tt.Normalize(*stats,inplace=True)])
valid_tfms = tt.Compose([tt.Resize((32,32)),tt.ToTensor(), tt.Normalize(*stats)])


In [ ]:
# PyTorch data loaders
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=3, pin_memory=True)
valid_dl = DataLoader(valid_ds, batch_size*2, num_workers=3, pin_memory=True)
test_dl = DataLoader(test_ds, batch_size*2, num_workers=3, pin_memory=True)

In [5]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [6]:
device = get_default_device()
device

device(type='cuda')

In [7]:
class ResNet9(nn.Module):
    def __init__(self):
        super(ResNet9, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
        )
        
        #         self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
#         self.fc = nn.Linear(256, num_classes)
        
        self.fc = nn.Sequential(nn.MaxPool2d(4), 
                      nn.Flatten(), 
                      nn.Dropout(0.2),
                      nn.Linear(256, num_classes))
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
#         x = self.avgpool(x)
#         x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [8]:
class SmallResNet9(nn.Module):
    def __init__(self, num_classes=3):
        super(SmallResNet9, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True)
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True)
        )

#         self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
#         self.fc = nn.Linear(256, num_classes)
        
        self.fc = nn.Sequential(nn.MaxPool2d(4), 
                      nn.Flatten(), 
                      nn.Dropout(0.2),
                      nn.Linear(256, num_classes))

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
#         x = self.avgpool(x)
#         x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [9]:
model = to_device(SmallResNet9(),device)

In [10]:
batch_size = 64

In [11]:
# Define dataset and dataloader
dataset = ImageFolder('./datasets/Datasets/Datasets/train', train_tfms)
dataloader = DataLoader(dataset, batch_size, shuffle=True, num_workers=3, pin_memory=True)
# dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
dataloader = DeviceDataLoader(dataloader, device)

In [12]:
k = 3
number_of_epochs = 50
learning_rate = 0.001

In [14]:
# Define training loop function
def train(model, train_loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    return epoch_loss

In [15]:
# Define evaluation function
def evaluate(model, val_loader):
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_labels.append(labels.cpu().numpy())
            all_preds.append(preds.cpu().numpy())
    all_labels = np.concatenate(all_labels)
    all_preds = np.concatenate(all_preds)
    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy

In [17]:
# Define k-fold cross-validation

kf = KFold(n_splits=k, shuffle=True)

# Initialize lists to store training and validation losses and accuracies


# Train and validate for each fold
for fold, (train_indices, val_indices) in enumerate(kf.split(dataset)):
    print(f'Fold {fold + 1}/{k}')
    
    # Create subset datasets and dataloaders for this fold
    train_subset = Subset(dataset, train_indices)
    val_subset = Subset(dataset, val_indices)
    train_loader = DataLoader(train_subset, batch_size, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size, shuffle=False)
    
    # Define model,
    model = SmallResNet9()
    model.to(device)
    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), learning_rate)
    
    train_losses = []
    val_losses = []
    val_accuracies = []
    
    # Train and validate for this fold
    for epoch in range(number_of_epochs):
        train_loss = train(model, train_loader, criterion, optimizer)
#         val_loss = evaluate(model, val_loader)
        val_loss = train(model,val_loader,criterion,optimizer)
        val_accuracy = evaluate(model, val_loader)
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)
        print(f'Epoch {epoch + 1} - Train loss: {train_loss:.4f} - Val loss: {val_loss:.4f} - Val accuracy: {val_accuracy:.4f}')
    
    # Save the model for this fold
    torch.save(model.state_dict(), f'resnet9_fold{fold}.pt')
    
# Print overall validation accuracy
print(f'Overall validation accuracy: {np.mean(val_accuracies):.4f}')


Fold 1/3
Epoch 1 - Train loss: 0.4687 - Val loss: 1.8222 - Val accuracy: 0.3026
Epoch 2 - Train loss: 1.2330 - Val loss: 1.6246 - Val accuracy: 0.3403
Epoch 3 - Train loss: 1.0973 - Val loss: 1.5592 - Val accuracy: 0.3534
Epoch 4 - Train loss: 1.0699 - Val loss: 1.4689 - Val accuracy: 0.3702
Epoch 5 - Train loss: 1.0090 - Val loss: 1.5601 - Val accuracy: 0.3597
Epoch 6 - Train loss: 0.9887 - Val loss: 1.5297 - Val accuracy: 0.4084
Epoch 7 - Train loss: 0.9282 - Val loss: 1.6198 - Val accuracy: 0.3686
Epoch 8 - Train loss: 0.9270 - Val loss: 1.5177 - Val accuracy: 0.3571
Epoch 9 - Train loss: 0.8214 - Val loss: 1.5337 - Val accuracy: 0.5901
Epoch 10 - Train loss: 0.7711 - Val loss: 1.5945 - Val accuracy: 0.4660
Epoch 11 - Train loss: 0.7656 - Val loss: 1.5151 - Val accuracy: 0.4560
Epoch 12 - Train loss: 0.7887 - Val loss: 1.4829 - Val accuracy: 0.4272
Epoch 13 - Train loss: 0.6959 - Val loss: 1.4454 - Val accuracy: 0.5340
Epoch 14 - Train loss: 0.5382 - Val loss: 1.4758 - Val accuracy:

KeyboardInterrupt: 

In [ ]:
torch.save(model, './saved_model.pth')

In [ ]:
def plot_losses():
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs');

In [ ]:
plot_losses()